# Learning graph
First we will learn the graph and then we will be making use of that learned graph to assign labels to unlabelled data

In [19]:
import numpy as np
import pandas as pd
from scipy import spatial
from sklearn.preprocessing import normalize
from scipy import sparse
from pyunlocbox import functions, solvers
from scipy import spatial
from matplotlib import pyplot as pl

In [20]:
df = pd.read_csv('processed.csv')
df.head()

,Unnamed: 0,variance,skewness,curtosis,entropy,Class,Class
0,0,3.62160,8.6661,-2.8073,-0.44699,0,NaN
1,1,4.54590,8.1674,-2.4586,-1.46210,0,NaN
2,2,3.86600,-2.6383,1.9242,0.10645,0,NaN
3,3,3.45660,9.5228,-4.0112,-3.59440,0,NaN
4,4,0.32924,-4.4552,4.5718,-0.98880,0,NaN


In [22]:
df.isnull().sum()

Unnamed: 0       0
variance         0
skewness         0
curtosis         0
entropy          0
Class            0
Class         1372
dtype: int64

In [24]:
df_new = df.drop('Class',axis=1)
X = df_new.to_numpy()
X_norm = normalize(X, axis=0, norm='max')

,Unnamed: 0,variance,skewness,curtosis,entropy,Class
0,0,3.62160,8.6661,-2.8073,-0.44699,0
1,1,4.54590,8.1674,-2.4586,-1.46210,0
2,2,3.86600,-2.6383,1.9242,0.10645,0
3,3,3.45660,9.5228,-4.0112,-3.59440,0
4,4,0.32924,-4.4552,4.5718,-0.98880,0


## Learning graph using the same code as Q1
Implementation of the paper : Kalofolias, V. ["How to learn a graph from smooth signals"](https://arxiv.org/abs/1601.02513), AISTATS, 2016

Logic credits : https://github.com/rodrigo-pena/graph-learning which is under MIT License

In [9]:
def mapping(n):
    #Total number of edges
    ne = int(n*(n-1)/2)
    r1 = np.zeros((ne, ))
    r2 = np.zeros((ne, ))
    itr = 0
    for i in np.arange(1, n):
        r1[itr: (itr + (n - i))] = i - 1
        r2[itr: (itr + (n - i))] = np.arange(i, n)
        itr = itr + n - i
    row = np.concatenate((r1, r2))
    col = np.concatenate((np.arange(0, ne), np.arange(0, ne)))
    values = np.ones(len(row))
    K = sparse.coo_matrix((values, (row, col)), shape=(n, ne))
    return lambda w: K.dot(w), lambda d: K.transpose().dot(d)

def learngraph(X,alpha=1,beta=1,step=0.5,maxit=1000, rtol=1e-5):
    N = X.shape[0]
    z = spatial.distance.pdist(X,'euclidean') # calculating pairwise distance
    w0 = np.zeros(z.shape)
    K, Kt = mapping(N)
    norm_K = np.sqrt(2 * (N - 1))
    
    # Assemble functions in the objective
    f1 = functions.func()
    f1._eval = lambda w: 2 * np.dot(w, z)
    f1._prox = lambda w, gamma: np.maximum(0, w - (2 * gamma * z))

    f2 = functions.func()
    f2._eval = lambda w: - alpha * np.sum(np.log(np.maximum(
        np.finfo(np.float64).eps, K(w))))
    f2._prox = lambda d, gamma: np.maximum(
        0, 0.5 * (d + np.sqrt(d**2 + (4 * alpha * gamma))))

    f3 = functions.func()
    f3._eval = lambda w: beta * np.sum(w**2)
    f3._grad = lambda w: 2 * beta * w
    lipg = 2 * beta
    
    # Rescale stepsize
    stepsize = step / (1 + lipg + norm_K)

    # Solve problem
    solver = solvers.mlfbf(L=K, Lt=Kt, step=stepsize)
    problem = solvers.solve([f1, f2, f3], x0=w0, solver=solver, maxit=maxit,
                            rtol=rtol)

    # Transform weight matrix from vector form to matrix form
    W = spatial.distance.squareform(problem['sol'])
    W[W<0] = 0
    return W

In [16]:
beta=5
alpha = 1
W = learngraph(X_norm,beta = beta,alpha = 1)

Solution found after 105 iterations:
    objective function f(sol) = -1.008477e+02
    stopping criterion: RTOL


In [17]:
print(W.shape)
#print(np.around(W))
print(W)

(1372, 1372)
[[0.00000000e+00 5.42487216e-02 2.05675805e-07 ... 0.00000000e+00
  5.92200903e-07 2.22733241e-07]
 [5.42487216e-02 0.00000000e+00 1.61711088e-08 ... 0.00000000e+00
  4.02696206e-07 3.32285439e-08]
 [2.05675805e-07 1.61711088e-08 0.00000000e+00 ... 0.00000000e+00
  5.89372787e-07 2.19905124e-07]
 ...
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  1.35081142e-02 0.00000000e+00]
 [5.92200903e-07 4.02696206e-07 5.89372787e-07 ... 1.35081142e-02
  0.00000000e+00 6.06430222e-07]
 [2.22733241e-07 3.32285439e-08 2.19905124e-07 ... 0.00000000e+00
  6.06430222e-07 0.00000000e+00]]
